In [1]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision.models import vgg19
import torchvision.transforms as transforms
import os.path as osp
from glob import glob
import shutil
from tqdm import tqdm

import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import sys
import json
import random

from torch import optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision.utils import save_image

In [2]:
"""
ディレクトリ 構成
root
    | - input
    |        | - annotation
    |        | - cat_face
    |                   | - demo
    |                   | - test
    |                   | - train
    |        | - images
    |                   | - cat
    |
    | - outputs

"""
#

'\nディレクトリ 構成\nroot\n    | - input\n    |        | - annotation\n    |        | - cat_face\n    |                   | - demo\n    |                   | - test\n    |                   | - train\n    |        | - images\n    |                   | - cat\n    |\n    | - outputs\n\n'

# データのダウンロード

In [3]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!tar xf 102flowers.tgz
!mkdir oxford-102
!mkdir oxford-102-train
!mkdir oxford-102-test
!mkdir params
!mkdir oxford-102/jpg
!mv jpg/*.jpg oxford-102/jpg
!cp ./oxford-102/jpg/image_0000[1-5].jpg ./oxford-102-test/

--2021-06-29 17:37:00--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
www.robots.ox.ac.uk (www.robots.ox.ac.uk) をDNSに問いあわせています... 129.67.94.2
www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 301 Moved Permanently
場所: https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz [続く]
--2021-06-29 17:37:01--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 344862509 (329M) [application/x-gzip]
`102flowers.tgz.1' に保存中

102flowers.tgz.1    100%[===================>] 328.89M  16.8MB/s    in 20s     

2021-06-29 17:37:21 (16.5 MB/s) - `102flowers.tgz.1' へ保存完了 [344862509/344862509]

mkdir: ディレクトリ `oxford-102' を作成できません: ファイルが存在します
mkdir: ディレクトリ `oxford-102-train' を作成できません: ファイルが存在します
mkdir: ディレクトリ `oxford-102-test' を作成できません: ファイルが存在します
mkdir: ディレクトリ `params' を作成できません: ファイルが

In [4]:
#パスの定義
image_dir = "./oxford-102/jpg/"
train_dir = "./oxford-102-train"
test_dir = "./oxford-102-test"
weight_save_dir = "./params/"
param_save_path = osp.join("./params/", 'param.json')
!mkdir ./output
image_test_save_dir = "./output"

mkdir: ディレクトリ `./output' を作成できません: ファイルが存在します


# データの加工

In [5]:
random_crop_times = 4
hr_const = 256
# クロップする画像のサイズ
crop_size = (hr_const, hr_const)

def random_crop(image, crop_size):
    h, w, _ = image.shape

    top = np.random.randint(0, h - crop_size[0])
    left = np.random.randint(0, w - crop_size[1])

    bottom = top + crop_size[0]
    right = left + crop_size[1]

    image = image[top:bottom, left:right, :]
    return image

# 学習に用いる画像
lst = [ "image_{:05}".format(i) for i in range(1,8189)]
cropped_image_save_path = "./oxford-102-train"

for item in tqdm(lst, total=len(lst)):
    image_name = '{}.jpg'.format(item)
    image_path = osp.join(image_dir, image_name)
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    # 画像のサイズが小さい時は対象から除外する。
    if (h < crop_size[0]) | (w < crop_size[1]):
        print('{} size is invalid. h: {},  w: {}'.format(image_name, h, w))
        continue
    #ランダムクロップ分だけ作る
    for num in range(random_crop_times):
        cropped_image = random_crop(image, crop_size=crop_size)
        image_save_name = '{}_{:03}.jpg'.format(item, num)
        cropped_image_save_path = osp.join(train_dir, image_save_name)
        os.makedirs(osp.dirname(cropped_image_save_path), exist_ok=True)
        cv2.imwrite(cropped_image_save_path, cropped_image)

100%|██████████| 8188/8188 [00:50<00:00, 161.51it/s]


# テスト用とデモ用の画像を専用のディレクトリに移しておく。

In [6]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [7]:
class ImageDataset(Dataset):
    """
    学習のためのDatasetクラス
    低解像度と高解像度の両者を提供する。
    """
    def __init__(self, dataset_dir, hr_shape):
        hr_height, hr_width = hr_shape
        
        # 低解像度の画像を取得するための処理
        self.lr_transform = transforms.Compose([
            transforms.Resize((hr_height // 4, hr_height // 4), Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])

        # 高像度の画像を取得するための処理
        self.hr_transform = transforms.Compose([
            transforms.Resize((hr_height, hr_height), Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])
        
        self.files = sorted(glob(osp.join(dataset_dir, '*')))
    
    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img_lr = self.lr_transform(img)
        img_hr = self.hr_transform(img)
        
        return {'lr': img_lr, 'hr': img_hr}
    
    def __len__(self):
        return len(self.files)

In [8]:
class TestImageDataset(Dataset):
    def __init__(self, dataset_dir):
        self.hr_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean, std)])
        self.files = sorted(glob(osp.join(dataset_dir, '*')))
    
    def lr_transform(self, img, img_size):
        img_width, img_height = img_size
        self.__lr_transform = transforms.Compose([
            transforms.Resize((img_height // 4, 
                               img_width // 4), 
                               Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)])
        img = self.__lr_transform(img)
        return img
            
    def __getitem__(self, index):
        img = Image.open(self.files[index % len(self.files)])
        img_size = img.size
        img_lr = self.lr_transform(img, img_size)
        img_hr = self.hr_transform(img)        
        return {'lr': img_lr, 'hr': img_hr}
    
    def __len__(self):
        return len(self.files)

# 生成器の準備

In [9]:
class DenseResidualBlock(nn.Module):
    def __init__(self, filters, res_scale = 0.2):
        super().__init__()
        self.res_scale = res_scale
        
        def block(in_features, non_linearity = True):
            layers = [nn.Conv2d(in_features, filters, 3, 1, 1, bias = True)]
            if non_linearity:
                layers += [nn.LeakyReLU()]
            return nn.Sequential(*layers)
    
        self.b1 = block(in_features = 1 * filters)
        self.b2 = block(in_features = 2 * filters)
        self.b3 = block(in_features = 3 * filters)
        self.b4 = block(in_features = 4 * filters)
        self.b5 = block(in_features = 5 * filters, non_linearity = False)
        self.blocks = [self.b1, self.b2, self.b3, self.b4, self.b5]
    
    def forward(self, x):
        inputs = x
        for block in self.blocks:
            out = block(inputs)
            inputs = torch.cat([inputs, out], dim = 1)
        return out.mul(self.res_scale) + x

In [10]:
# サイズの確認
#x = torch.randn(10, 64, 5, 5)
#D = DenseResidualBlock(filters = 64)
#D(x).shape

In [11]:
class ResidualInResidualDenseBlock(nn.Module):
    def __init__(self, filters, res_scale=0.2):
        super().__init__()
        self.res_scale = res_scale
        self.dense_blocks = nn.Sequential(
            DenseResidualBlock(filters), 
            DenseResidualBlock(filters), 
            DenseResidualBlock(filters)
        )
    
    def forward(self, x):
        return self.dense_blocks(x).mul(self.res_scale) + x

In [12]:
# サイズの確認
#DD = ResidualInResidualDenseBlock(filters = 64)
#DD(x).shape

In [13]:
"""
層の構造

Conv2D
RinRD-Block
     .
     .
RinRD-Block
Conv2D
|---------------|
|Conve2D     |
|LeaklyReLU |
|Pixelshuffle |
|---------------|
     .
     .
|---------------|
|Conve2D     |
|LeaklyReLU |
|Pixelshuffle |
|---------------|
Conv2D
LeaklyReLU
Conv2D
"""

class GeneratorRRDB(nn.Module):
    def __init__(self, channels, filters = 64, num_res_blocks = 16, num_upsample = 2):
        super().__init__()
        
        self.conv1 = nn.Conv2d(channels, filters, kernel_size = 3, stride = 1, padding = 1)
        self.res_blocks = nn.Sequential(*[ResidualInResidualDenseBlock(filters)  for _ in range(num_res_blocks)])
        self.conv2 = nn.Conv2d(filters, filters, kernel_size = 3, stride = 1, padding = 1)
        
        upsample_layers = []
        
        for _ in range(num_upsample):
            upsample_layers += [
                nn.Conv2d(filters, filters * 4, kernel_size = 3, stride = 1, padding = 1),
                nn.LeakyReLU(),
                nn.PixelShuffle(upscale_factor = 2),
            ]
        self.upsampling = nn.Sequential(*upsample_layers)
        self.conv3 = nn.Sequential(
            nn.Conv2d(filters, filters, kernel_size = 3, stride = 1, padding = 1),
            nn.LeakyReLU(),
            nn.Conv2d(filters, channels, kernel_size = 3, stride = 1, padding = 1),
        )
    
    def forward(self, x):
        out1 = self.conv1(x)
        #print(1,out1.shape)
        out = self.res_blocks(out1)
        #print(2,out.shape)
        out2 = self.conv2(out)
        #print(3,out2.shape)
        out = torch.add(out1, out2)
        #print(4,out.shape)
        out = self.upsampling(out)
        #print(5,out.shape)
        out = self.conv3(out)
        #print(6,out.shape)
        return out

In [14]:
# サイズの確認
# ex : (10,3,32,32) --> (10, 3, 128, 128)
#y = torch.randn(10,3,32,32)
#DDD = GeneratorRRDB(3, filters = 64, num_res_blocks = 23)
#DDD(y).shape

# 特徴量抽出器

In [15]:
class FeatureExtractor(nn.Module):
    """
    vgg19を応用した特徴量抽出器
    """
    def __init__(self):
        super().__init__()
        vgg19_model = vgg19(pretrained = True)
        self.vgg19_54 = nn.Sequential(*list(vgg19_model.features.children())[:35])

    def forward(self, x):
        return self.vgg19_54(x)

In [16]:
# サイズ確認用
# ex : (10, 3, 128, 128) --> (10, 512, 8,8)
#y = torch.randn(10,3,32,32)
#FE = FeatureExtractor()
#FE(y).shape

# 識別器の準備

In [17]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
                
        self.input_shape = input_shape
        in_channels, in_height, in_width = self.input_shape
        patch_h, patch_w = int(in_height / 2 ** 4), int(in_width / 2 ** 4)
        self.output_shape = (1, patch_h, patch_w)
    
        def discriminator_block(in_filters, out_filters, first_block = False):
            layers = []
            layers.append(nn.Conv2d(in_filters, 
                                    out_filters, 
                                    kernel_size = 3, 
                                    stride = 1, 
                                    padding = 1))
            if not first_block:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace = True))
            layers.append(nn.Conv2d(out_filters, 
                                    out_filters, 
                                    kernel_size = 3, 
                                    stride = 2, 
                                    padding = 1))
            layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace = True))
            return layers
        
        layers = []
        in_filters = in_channels
        for i, out_filters in enumerate([64, 128, 256, 512]):
            #print(discriminator_block(in_filters, out_filters,  first_block=(i == 0)))
            layers.extend(discriminator_block(in_filters, out_filters, first_block=(i == 0)))
            in_filters = out_filters
        
        layers.append(nn.Conv2d(out_filters,  1, kernel_size = 3, stride = 1,  padding = 1))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, img):
        return self.model(img)

# ESRGANクラスの構成

In [18]:
class ESRGAN():
    def __init__(self, opt):
        # 生成器・識別器の設定
        self.generator = GeneratorRRDB(opt.channels, filters = 64, num_res_blocks = opt.residual_blocks).to(opt.device)
        self.discriminator = Discriminator(input_shape = (opt.channels, *hr_shape)).to(opt.device)

        # 特徴量抽出器の設定
        self.feature_extractor = FeatureExtractor().to(opt.device)
        self.feature_extractor.eval()
        
        # 損失関数の設定
        self.criterion_GAN = nn.BCEWithLogitsLoss().to(opt.device)
        self.criterion_content = nn.L1Loss().to(opt.device)
        self.criterion_pixel = nn.L1Loss().to(opt.device)

        # オプティマイザーの設定
        self.optimizer_G = optim.Adam(self.generator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))
        self.optimizer_D = optim.Adam(self.discriminator.parameters(), lr = opt.lr, betas = (opt.b1, opt.b2))

        # デバイスとテンソル型の定義
        self.Tensor = torch.Tensor
        self.dev = opt.device
        
    #==================================================================================
    
    def pre_train(self, imgs, batch_num):
        imgs_lr = imgs['lr'].type(torch.Tensor).to(self.dev)
        imgs_hr = imgs['hr'].type(torch.Tensor).to(self.dev)

        valid = torch.tensor(np.ones((imgs_lr.size(0), *self.discriminator.output_shape)), requires_grad=False).to(self.dev)
        fake = torch.tensor(np.zeros((imgs_lr.size(0), *self.discriminator.output_shape)), requires_grad=False).to(self.dev)

        # 勾配初期化
        self.optimizer_G.zero_grad()

        # 低解像度 --> 高解像度を実行し、ピクセル単位の損失計算
        gen_hr = self.generator(imgs_lr)
        loss_pixel = self.criterion_pixel(gen_hr, imgs_hr)

        # 画素単位の損失であるloss_pixelで事前学習を行う
        loss_pixel.backward()
        self.optimizer_G.step()
        train_info = {'epoch': epoch, 'batch_num': batch_num, 'loss_pixel': loss_pixel.item()}
        """
        if batch_num == 1:
            sys.stdout.write('\n{}'.format(train_info))
        else:
            sys.stdout.write('\r{}'.format('\t'*20))
            sys.stdout.write('\r{}'.format(train_info))
        """
        
        sys.stdout.write('\r{}'.format('\t'*20))
        sys.stdout.write('\r{}'.format(train_info))
        sys.stdout.flush()
        
    #============================================================================
    
    def train(self, imgs, batch_num):
        imgs_lr = imgs['lr'].type(self.Tensor).to(self.dev)
        imgs_hr = imgs['hr'].type(self.Tensor).to(self.dev)

        ################### 
        #####生成器の損失#####
        ###################
        # 正解ラベル
        valid = torch.tensor(np.ones((imgs_lr.size(0), *self.discriminator.output_shape)), requires_grad = False).to(self.dev)
        fake = torch.tensor(np.zeros((imgs_lr.size(0), *self.discriminator.output_shape)),requires_grad = False).to(self.dev)

        # 低解像度 --> 高解像度
        self.optimizer_G.zero_grad()
        gen_hr = self.generator(imgs_lr)

        # (1)ピクセル単位の損失計算
        loss_pixel = self.criterion_pixel(gen_hr, imgs_hr)

        # (2)Adversarial loss
        # 本物画像と超解像画像の識別器による判定を見る
        pred_real = self.discriminator(imgs_hr).detach()
        pred_fake = self.discriminator(gen_hr)
        loss_GAN = self.criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), valid)

        # (3)Perceptual loss
        #特徴量の比較、シンプルな損失関数
        gen_feature = self.feature_extractor(gen_hr)
        real_feature = self.feature_extractor(imgs_hr).detach()
        loss_content = self.criterion_content(gen_feature, real_feature)

        
        loss_G = loss_content + opt.lambda_adv * loss_GAN + opt.lambda_pixel * loss_pixel
        loss_G.backward()
        self.optimizer_G.step()

        
        ################### 
        #####識別機の損失#####
        ###################
        self.optimizer_D.zero_grad()
        pred_real = self.discriminator(imgs_hr)
        pred_fake = self.discriminator(gen_hr.detach())

        loss_real = self.criterion_GAN(pred_real - pred_fake.mean(0, keepdim=True), valid)            
        loss_fake = self.criterion_GAN(pred_fake - pred_real.mean(0, keepdim=True), fake)    
        loss_D = (loss_real + loss_fake) / 2
        loss_D.backward()
        self.optimizer_D.step()

        train_info = {'epoch': epoch, 'batch_num': batch_num,  'loss_D': loss_D.item(), 'loss_G': loss_G.item(),
                      'loss_content': loss_content.item(), 'loss_GAN': loss_GAN.item(), 'loss_pixel': loss_pixel.item(),}
        """
        if batch_num == 1:
            sys.stdout.write('\n{}'.format(train_info))
        else:
            sys.stdout.write('\r{}'.format('\t'*20))
            sys.stdout.write('\r{}'.format(train_info))
        """
        sys.stdout.write('\r{}'.format('\t'*20))
        sys.stdout.write('\r{}'.format(train_info))
        sys.stdout.flush()

    def save_image(self,imgs,batches_done):
        with torch.no_grad():
            imgs_lr = imgs["lr"].type(self.Tensor).to(self.dev)
            gen_hr = self.generator(imgs_lr)
            gen_hr = denormalize(gen_hr)

            image_batch_save_dir = osp.join(image_test_save_dir, '{:03}'.format(i))
            gen_hr_dir = osp.join(image_batch_save_dir, "hr_image")
            os.makedirs(image_batch_save_dir, exist_ok=True)
            save_image(gen_hr, osp.join(image_batch_save_dir, "{:09}.png".format(batches_done)), nrow=1, normalize=False)
        

    def save_weight(self, batches_done):
        """
        重みの保存
        """
        generator_weight_path = osp.join(weight_save_dir, "generator_{:08}.pth".format(batches_done))
        discriminator_weight_path = osp.join(weight_save_dir, "discriminator_{:08}.pth".format(batches_done))

        torch.save(self.generator.state_dict(), generator_weight_path)
        torch.save(self.discriminator.state_dict(), discriminator_weight_path)

# 出力用の関数などの定義

In [19]:
def denormalize(tensors):
    """
    平均と標準偏差により出力を加工
    """
    for chn in range(3):
        tensors[:, chn].mul_(std[chn]).add_(mean[chn])
    return torch.clamp(tensors, 0, 255)

def save_json(file, save_path, mode):
    """
    Jsonファイルを保存
    """
    with open(save_path, mode) as outfile:
        json.dump(file, outfile, indent=4)

# パラメータの準備

In [20]:
#パラメータセット
class Opts():
    def __init__(self):
        self.n_epoch = 50
        self.residual_blocks = 17
        self.lr = 0.0002
        self.b1 = 0.9
        self.b2 = 0.999
        self.batch_size = 16
        self.n_cpu = 8
        self.warmup_batches = 500
        self.lambda_adv = 5e-3
        self.lambda_pixel = 1e-2
        self.pretrained = False
        self.dataset_name = 'cat'
        self.sample_interval = 100
        self.checkpoint_interval = 1000
        self.hr_height = hr_const
        self.hr_width = hr_const
        self.channels = 3
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print(self.device)
    def to_dict(self):
        parameters = {
            'n_epoch': self.n_epoch,
            'hr_height': self.hr_height,
            'residual_blocks': self.residual_blocks,
            'lr': self.lr,
            'b1': self.b1,
            'b2': self.b2,
            'batch_size': self.batch_size,
            'n_cpu': self.n_cpu,
            'warmup_batches': self.warmup_batches,
            'lambda_adv': self.lambda_adv,
            'lambda_pixel': self.lambda_pixel,
            'pretrained': self.pretrained,
            'dataset_name': self.dataset_name,
            'sample_interval': self.sample_interval,
            'checkpoint_interval': self.checkpoint_interval,
            'hr_height': self.hr_height,
            'hr_width': self.hr_width,
            'channels': self.channels,
            'device': str(self.device),
        }
        return parameters
opt = Opts()
save_json(opt.to_dict(), param_save_path, 'w')
hr_shape = (opt.hr_height, opt.hr_height)

cuda:0


# データローダーの準備

In [21]:
train_dataloader = DataLoader(
    ImageDataset(train_dir, hr_shape = hr_shape),
    batch_size = opt.batch_size,
    shuffle = True,
    num_workers = opt.n_cpu,
)

test_dataloader = DataLoader(
    TestImageDataset(test_dir),
    batch_size = 1,
    shuffle = False,
    num_workers = opt.n_cpu,
)

# 学習開始

In [22]:
esrgan = ESRGAN(opt)

In [23]:
for epoch in range(1, opt.n_epoch + 1):
    print("\ncurrent epoch : ", epoch)
    for batch_num, imgs in enumerate(train_dataloader):
        if batch_num < 1000:
            batches_done = (epoch - 1) * len(train_dataloader) + batch_num
            # 事前学習
            if batches_done <= opt.warmup_batches:#500
                esrgan.pre_train(imgs, batch_num)
            # メイン学習
            else:
                esrgan.train(imgs, batch_num)
            """
            # 高解像度の生成画像の保存
            if batches_done % opt.sample_interval == 0:
                for i, imgs in enumerate(test_dataloader):
                    esrgan.save_image(imgs, batches_done)
            """
            if batches_done % opt.sample_interval == 0:
                for i, imgs in enumerate(test_dataloader):
                    esrgan.save_image(imgs,batches_done)
            # 学習した重みの保存
            if batches_done % opt.checkpoint_interval == 0:
                esrgan.save_weight(batches_done)
        else:
            continue


current epoch :  1
{'epoch': 1, 'batch_num': 999, 'loss_D': 0.02499145880246073, 'loss_G': 0.9530349186949307, 'loss_content': 0.9277909994125366, 'loss_GAN': 4.819658363272741, 'loss_pixel': 0.1145627498626709}}}}
current epoch :  2
{'epoch': 2, 'batch_num': 999, 'loss_D': 0.05047074742428714, 'loss_G': 0.9071336459487345, 'loss_content': 0.8837738633155823, 'loss_GAN': 4.405306091115686, 'loss_pixel': 0.1333252191543579}}}}}}
current epoch :  3
{'epoch': 3, 'batch_num': 999, 'loss_D': 0.07522459701344696, 'loss_G': 0.8985897478938538, 'loss_content': 0.8762665390968323, 'loss_GAN': 4.252139349700284, 'loss_pixel': 0.10625120252370834}}}}
current epoch :  4
{'epoch': 4, 'batch_num': 999, 'loss_D': 0.0024322768677695157, 'loss_G': 0.8379886678377648, 'loss_content': 0.7972395420074463, 'loss_GAN': 7.896028984422182, 'loss_pixel': 0.12689809501171112}6}
current epoch :  5
{'epoch': 5, 'batch_num': 999, 'loss_D': 0.00016243845968615372, 'loss_G': 0.7638726389999316, 'loss_content': 0.71

{'epoch': 38, 'batch_num': 999, 'loss_D': 0.08748236421455058, 'loss_G': 0.6547391521518816, 'loss_content': 0.6372362375259399, 'loss_GAN': 3.326710588443074, 'loss_pixel': 0.08693616837263107}6}}}
current epoch :  39
{'epoch': 39, 'batch_num': 999, 'loss_D': 0.06314885156047935, 'loss_G': 0.7383007730915776, 'loss_content': 0.697236955165863, 'loss_GAN': 7.973049210377163, 'loss_pixel': 0.11985718458890915}23}}}
current epoch :  40
{'epoch': 40, 'batch_num': 999, 'loss_D': 0.010372030576885922, 'loss_G': 0.7504492440032204, 'loss_content': 0.7167840600013733, 'loss_GAN': 6.516064810648376, 'loss_pixel': 0.10848599672317505}}}}
current epoch :  41
{'epoch': 41, 'batch_num': 999, 'loss_D': 0.003053087491426013, 'loss_G': 0.7594229220915535, 'loss_content': 0.7157421112060547, 'loss_GAN': 8.473323644882921, 'loss_pixel': 0.13141927123069763}}}}
current epoch :  42
{'epoch': 42, 'batch_num': 999, 'loss_D': 0.00466359841341872, 'loss_G': 0.6125866900511955, 'loss_content': 0.5770354270935

# GANの準備

In [24]:
esrgan = GeneratorRRDB(3, filters = 64, num_res_blocks = 17).to("cuda:0")
esrgan.load_state_dict(torch.load("./params/generator_00101000.pth"))
esrgan.to("cpu")
for p in esrgan.parameters():
    p.requires_grad = False
print()

In [25]:
batch_size = 64
dev = "cuda:0"

In [26]:
nz = 100
ngf = 32
#(100,1,1) --> (3,64,64)
class GNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz,ngf*8,4,1,0,bias=False),
            nn.BatchNorm2d(ngf*8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*8,ngf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*4,ngf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*2,ngf*1,4,2,1,bias=False),
            nn.BatchNorm2d(ngf*1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(ngf*1,3,4,2,1,bias=True),
            nn.Tanh()
            )
    def forward(self,x):
        out = self.main(x)
        return out

In [27]:
ndf=32
#(3,64,64) --> (1)
class DNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3,ndf,4,2,1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf,ndf*2,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*2,ndf*4,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*4,ndf*8,4,2,1,bias=False),
            nn.BatchNorm2d(ndf*8),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf*8,1,4,1,0,bias=True)
            )

    def forward(self,x):
        out = self.main(x)
        return out.squeeze()

In [28]:
d = DNet().to(dev)
g = GNet().to(dev)

opt_d = optim.Adam(d.parameters(),lr=0.0002,betas=(0.5,0.999))
opt_g = optim.Adam(g.parameters(),lr=0.0002,betas=(0.5,0.999))

ones = torch.ones(batch_size).to(dev)
zeros = torch.zeros(batch_size).to(dev)

loss_f = nn.BCEWithLogitsLoss()

fixed_z = torch.randn(64,nz,1,1).to(dev)

In [29]:
from statistics import mean

def train_dcgan(g,d,opt_g,opt_d,loader):
    log_loss_g = []
    log_loss_d = []
    
    for real_img  in tqdm(loader):
        #save_image(real_img[0],"./data_tmp/tmp.jpg")
        real_img = real_img["lr"]
        batch_len = len(real_img)
        #print(real_img.size())
        real_img = real_img.to(dev)
        z = torch.randn(batch_len,nz,1,1).to(dev)
        #------------------------------------------------------------------------------------------------------------------
        fake_img = g(z)                                                         #フェイクイメージの作成
        fake_img_tensor = fake_img.detach()                  #フェイクイメージを計算グラフから分離
        out = d.forward(fake_img)                                      #フェイクイメージの判定
        loss_g = loss_f(out, ones[:batch_len])                 #フェイク画像の損失計算 as 本物
        log_loss_g.append(loss_g.item())                        #
        d.zero_grad() , g.zero_grad()                               #勾配初期化
        loss_g.backward()                                                  #生成機の逆伝搬
        opt_g.step()                                                           #生成機の更新
        #------------------------------------------------------------------------------------------------------------------
        real_out = d.forward(real_img)                                       #本物画像の判定
        loss_d_real = loss_f(real_out,ones[:batch_len])         #本物画像の損失計算

        fake_img = fake_img_tensor                                          #
        fake_out = d.forward(fake_img_tensor)                      #
        loss_d_fake = loss_f(fake_out, zeros[:batch_len])    #フェイク画像の損失計算 as フェイク

        loss_d = loss_d_real + loss_d_fake
        log_loss_d.append(loss_d.item())
        d.zero_grad(),g.zero_grad()                                         #勾配初期化
        loss_d.backward()                                                          #判別機の逆伝搬
        opt_d.step()                                                                   #判別機の更新
        #------------------------------------------------------------------------------------------------------------------
    return mean(log_loss_g),mean(log_loss_d)

In [30]:
if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists("data2"):
    os.mkdir("data2")
if not os.path.exists("data_tmp"):
    os.mkdir("data_tmp")

In [31]:
import time

total_time=0.0
average_time=0.0
iter_epoch=200


for epoch in range(iter_epoch):
    t_start =time.time()
    print("current epoch : ",epoch)
    #=====================================================================================
    train_dcgan(g,d,opt_g,opt_d,train_dataloader)
    if epoch%1==0:
        torch.save(g.state_dict(),"./data/g_{:03d}.prm".format(epoch),pickle_protocol=4)
        torch.save(d.state_dict(),"./data/d_{:03d}.prm".format(epoch),pickle_protocol=4)
        generated_img_tmp = g(fixed_z).to("cpu")
        generated_img = esrgan(generated_img_tmp)
        #g = g.to(dev)
        save_image(generated_img,"./data/{:03d}.jpg".format(epoch))
        save_image(generated_img_tmp,"./data2/{:03d}.jpg".format(epoch))
        del generated_img_tmp, generated_img
    #=====================================================================================
    t_finish=time.time()
    total_time+=t_finish-t_start
    average_time=total_time/(epoch+1)
    print("runtime = ",t_finish-t_start,"sec")
    print("expected remaining time = ",int(average_time*(iter_epoch-(epoch+1))),"sec",":",int(average_time*(iter_epoch-(epoch+1))/60),"minute")
    


files = sorted(glob('./data/*.jpg'))  
images = list(map(lambda file : Image.open(file) , files))
images[0].save('generating_process.gif' , save_all = True , append_images = images[1:] , duration = 200 , loop = 0)

  0%|          | 0/2047 [00:00<?, ?it/s]

current epoch :  0


  0%|          | 0/2047 [00:00<?, ?it/s]

runtime =  76.60332942008972 sec
expected remaining time =  15244 sec : 254 minute
current epoch :  1


  0%|          | 0/2047 [00:00<?, ?it/s]

...

runtime =  74.19910740852356 sec
expected remaining time =  149 sec : 2 minute
current epoch :  198


  0%|          | 0/2047 [00:00<?, ?it/s]

runtime =  74.44593024253845 sec
expected remaining time =  74 sec : 1 minute
current epoch :  199


100%|██████████| 2047/2047 [00:55<00:00, 36.85it/s]


runtime =  77.63209342956543 sec
expected remaining time =  0 sec : 0 minute
